In [36]:
import sqlite3
import hashlib
import datetime

In [10]:
dbname = ('users.db')
conn = sqlite3.connect(dbname, isolation_level=None)

In [11]:
cursor = conn.cursor()
sql = """CREATE TABLE IF NOT EXISTS userinfo(id, password)"""

cursor.execute(sql)
conn.commit()

sql = """CREATE TABLE IF NOT EXISTS user_stock(id, securities_code, amount, buy_date)"""
cursor.execute(sql)
conn.commit()

In [12]:
sql = """SELECT name FROM sqlite_master WHERE TYPE='table'"""

for t in cursor.execute(sql):
    print(t)

('userinfo',)
('user_stock',)


In [13]:
username = "user"
password = "Password"

password_hash = hashlib.sha256(password.encode()).hexdigest()

In [14]:
sql = """INSERT INTO userinfo VALUES(?, ?)"""

data = ((username, password_hash))
cursor.execute(sql, data)
conn.commit()

In [15]:
sql = """INSERT INTO userinfo VALUES(?,?)"""

data = [
    ("user2", hashlib.sha256("Password2".encode()).hexdigest()),
    ("user3", hashlib.sha256("Password3".encode()).hexdigest()),
    ("user4", hashlib.sha256("Password4".encode()).hexdigest()),
    ("user5", hashlib.sha256("Password5".encode()).hexdigest())
]

cursor.executemany(sql, data)
conn.commit()

In [16]:
sql = """SELECT * FROM userinfo"""
cursor.execute(sql)
print(cursor.fetchall())

[('user', 'e7cf3ef4f17c3999a94f2c6f612e8a888e5b1026878e4e19398b23bd38ec221a'), ('user2', '1be0222750aaf3889ab95b5d593ba12e4ff1046474702d6b4779f4b527305b23'), ('user3', '2538f153f36161c45c3c90afaa3f9ccc5b0fa5554c7c582efe67193abb2d5202'), ('user4', 'db514f5b3285acaa1ad28290f5fefc38f2761a1f297b1d24f8129dd64638825d'), ('user5', '8180d5783fea9f86479e748f6d2d1196c4a8e143643119398c16367d2c3d50f2')]


In [30]:
sql = """SELECT * FROM userinfo WHERE id = ?"""
###入力されたユーザーid
search_id = 'user'

cursor.execute(sql,[search_id])
fit = cursor.fetchall()
print(fit)
print(type(fit))
###ユーザーidとハッシュ関数にかけたパスワードがタプルで出力されるので、そのようなタプルを作成して比較する。

[('user', 'e7cf3ef4f17c3999a94f2c6f612e8a888e5b1026878e4e19398b23bd38ec221a')]
<class 'list'>


In [37]:
###入力されたデータが保存されているものと一緒であるかどうかを確認する。
received_pass = "Password"

recPass_hash = hashlib.sha256(received_pass.encode()).hexdigest()
user_tuple = (search_id, recPass_hash)

if fit[0] == user_tuple:
    print("確認ができました")
    sql = """SELECT * FROM user_stock WHERE id = ?"""
    cursor.execute(sql, [search_id])
    hit = cursor.fetchall()
    print(hit)
    ###購入した株の証券コードやその量、購入日などの情報がユーザー情報と一緒に格納されている。
    ###擬似的な購入手続きをいかに記す
    securities_code = 6758
    amount = 100
    date = datetime.datetime.now()
    if amount % 100 != 0:
        print("日本の株は100株が基本単位となっています。")
    else:
        data = ((search_id, securities_code, amount, date))
        sql = """INSERT INTO user_stock VALUES(?, ?, ?, ?)"""
        cursor.execute(sql, data)
        conn.commit()
        sql = """SELECT * FROM user_stock"""
        cursor.execute(sql)
        print(cursor.fetchall())
        
else:
    print("ユーザーIDかパスワードが間違っています")

確認ができました
[]
[('user', 6758, 100, '2023-05-17 10:48:43.758242')]


In [11]:
conn.close()